# Create Vaccine Location Walksheds
This notebook uses the [NYC Vaccine Location dataset](https://vaccinefinder.nyc.gov/locations) and creates a 15-minute walkshed isochrone for each location. This dataset defines a function to create an isochrone, loops through each vaccine location, and then writes the resulting isochrones to `iso.csv`. 

# Initialize Workspace

## Import Libraries

In [ ]:
# for data
import pandas as pd

# for spatial data
import geopandas as gpd

# for network analysis
import networkx as nx

# for street network analysis
import osmnx as ox

# to manipulate csv files
import csv
from csv import DictWriter

## Import Data
Bring in the vaccine locations from a geojson file.

In [ ]:
# read in the geojson
vac = gpd.read_file('data/vac.geojson')
vac = vac[0:2].reset_index().copy()

# Create Isochrones

## Set Up
Define variables and create an empty .csv for the isochrones. 

In [ ]:
# define variables for the function
network_type = 'walk' # create walkshed
meters_per_minute = 75 # travel distance per minute
time = 15 # minutes
color = '#228B22'

<div class="alert alert-danger">
Do NOT run this cell. It will overwrite the existing completed file. 
</div>

In [ ]:
# create blank csv file for isochrones

# name of csv file  
filename = "data/iso.csv"
    
# field names  
fields = ['time', 'geometry']

# writing to csv file ('w') 
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  

## Create Function
This function takes in a lat/lon tuple from the vaccine dataset, creates an isochrone, and then appends it to `iso.csv`.

In [ ]:
def create_iso(place, name):
    # download the street network
    G = ox.graph_from_point(place, dist = 2000, network_type=network_type)
    
    # project network data to Web Mercator (meters)
    G = ox.project_graph(G, to_crs='epsg:3857')
    
    # convert nodes and edges to geodataframes
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    
    # get the bounding box coordinates
    minx, miny, maxx, maxy = gdf_nodes.geometry.total_bounds
    
    # calculate the centroid
    centroid_x = (maxx-minx)/2 + minx
    centroid_y = (maxy-miny)/2 + miny
    
    # use osmnx's get_nearest_node command to get the id for the nearest node
    center_node = ox.get_nearest_node(G, 
                                  (centroid_y,centroid_x), 
                                  method = 'euclidean')
    
    # for each trip time, create an egograph of nodes that fall within that distance
    subgraph = nx.ego_graph(G, center_node, radius=time, distance='time')

    # for each of those nodes, update the gdf_nodes dataframe and assign it with its associated distance color
    for node in subgraph.nodes():
        gdf_nodes.loc[node,'time'] = str(time) + ' mins'
        gdf_nodes.loc[node,'color'] = color
        
    # the NaN values then need to be populated with a valid color
    gdf_nodes['color'].fillna('#cccccc', inplace=True)
    
    # dissolve the nodes by time
    isochrone = gdf_nodes.dissolve("time")
    
    # for each row, create a convex hull
    isochrone = isochrone.convex_hull.reset_index()
    
    # geometry header has been automatically named "0"
    isochrone.columns=['time','geometry']
    
    # name of csv file  
    filename = "data/iso_test.csv"
    
    # field names  
    fields = list(isochrone)
    
    with open('iso_test.csv', 'a') as isofile: 

            dictwriter = DictWriter(isofile, fieldnames=fields) 

            for index, row in isochrone.iterrows():
                dictwriter.writerow(row.to_dict()) 

            isofile.close()    
      

# Loop
Loop through each row in the vaccine dataset and use the `create_iso` function to generate an isochrone for each location. Note: This step takes 20+ minutes to run for the entire dataset, but does not overload the memory.

In [ ]:
# run loop
for i in range(len(vac)):
    place = (vac.Latitude[i], vac.Longitude[i])
    name = vac.FacilityName[i]
    create_iso(place, name)

In [ ]:
# export iso gdf as a geojson
iso.to_file("data/iso.geojson", driver='GeoJSON')

# Notes
The result of this notebook is `iso.csv`, which contains an isochrone for each row in the vaccine dataset. It will be used next in the "Accessibility Analysis" notebook.

In the first iteration, I used multiple `for` loops and stored multiple dataframes in dictionaries. However, this method required running the notebook six separate times in order to process in chunks that would not overload the memory. I've streamlined and simplified the code by using a function within a single `for` loop. In addition simplifying the code, using a function consumes far less memory because the intermediary dataframes are overwritten in each iteration, rather than attempting to store all the resulting dataframes from all iterations. The loop itself takes a significant amount of time to run, but I only need to run it once!

<p> This notebook was created by Lauren Harper.</p>